# Pydantic

## Какие проблемы решает?

Валидация данных необходима при их получении из стороних ресурсов. 
Например есть некоторый сервис. Он получает данные о пользователе из разных источников данных. У пользователя должны быть следующие данные:

In [ ]:
from dataclass import dataclass


@dataclass
class User:
    uuid: str
    name: str

## Библиотеки для валидации данных

Самые популярные библиотеки валидирующие данные для языка Python:
* Cerberus, Voluptuous, Schema (работа с этими библиотеками похожа)
* Marshmallow
* Pydantic

### Сerberus

In [ ]:
!pip install Cerberus

Предоставляет функционал для валидализации данных

Необходимо отпределить схему валидализации и передать ее в класс Validator. Схема является mapping, чаще всего используется dict

In [2]:
from cerberus import Validator

In [26]:
user_validator = Validator(
    {
        'name': {'type': 'string'},
        'email': {'type': 'string', 'required': True},
        'age': {'type': 'integer', 'min': 0, 'max': 150}
    }
)

In [28]:
user = {
    'name': 'foo',
    'email': 'foo@bar.com',
    'age': 30,
}
current_user = user_validator.validate(user)
current_user

True

Минусы:

    1. Свой собственный язык даных (str -> string, int -> integer)
    2. Тяжело описать сложную схему с помощью dict (пример  validate ipv4 + network interface)
    3. Сложные схемы проще описать в YAML файле
    4. Выполняет только валидацию данных, не возвращает объект экземпляр класса

### marshmallow

In [ ]:
!pip install marshmallow

Схема описывается в виде класса с помощью типов из fields.

In [59]:
from marshmallow import Schema, fields, validate


class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email(required=True)
    age = fields.Int()

In [60]:
user = {
    'name': 'foo',
    'email': 'foo@bar.com',
    'age': 30,
}
schema = UserSchema()
schema.dump(user)

{'email': 'foo@bar.com', 'name': 'foo', 'age': 30}

Необходимо реализовать еще один класс, чтобы получить экземпляр класса

In [51]:
from marshmallow import Schema, fields, validate, post_load, post_dump


class User:
    def __init__(self, name, email, age):
        self.name = name
        self.email = email
        self.age = age
        
    def __repr__(self):
        return f'name={self.name}, email={self.email}, age={self.age}'
        

class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email(required=True)
    age = fields.Int()
    
    @post_dump
    def make_user(self, data, **kwargs):
        return User(**data)

In [52]:
user = {
    'name': 'foo',
    'email': 'foo@bar.com',
    'age': 30,
}
schema = UserSchema()
schema.dump(user)

name=foo, email=foo@bar.com, age=30

Можно использовать вложенные схемы и структуры данных

In [53]:
from marshmallow import Schema, fields, validate


class BookSchema(Schema):
    title = fields.String(required=True, validate=validate.Length(max=120))
    isbn = fields.String(required=True)

class UserSchema(Schema):
    email = fields.Email(required=True)
    books = fields.Nested(BookSchema, many=True, required=False)

Плюсы:

    1. Схема задает в виде класса
    2. Нет необходимости изучать дополнительный язык типов данных. Все типы данных содержатся в fields и любая ide может их подсказать
    3. Можно возращать валидированный экземпляр класса
Минусы:

    1. Если нужно вернуть экземпляр класса, то необходимо реализовать два класса

### pydantic

In [1]:
!pip install pydantic

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


Объекты для валидации описываются с помощью классов, который возвращает экземпляр класса. Для описание типов используются стандартные типы Python.

In [2]:
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: Optional[str]
    email: str
    age: int

In [3]:
user = {
    'name': 'foo',
    'email': 'foo@bar.com',
    'age': 30,
}
User(**user)

User(name='foo', email='foo@bar.com', age=30)

parse_obj - this is very similar to the __init__ method of the model, except it takes a dict rather than keyword arguments. If the object passed is not a dict a ValidationError will be raised.

In [4]:
user = {
    'name': 'foo',
    'email': 'foo@bar.com',
    'age': 30,
}
User.parse_obj(user)

User(name='foo', email='foo@bar.com', age=30)

parse_raw -  this takes a str or bytes and parses it as json, then passes the result to

In [7]:
import json
user_json = json.dumps(user)
user_json

'{"name": "foo", "email": "foo@bar.com", "age": 30}'

In [8]:
User.parse_raw(user_json)

User(name='foo', email='foo@bar.com', age=30)

parse_file: this takes in a file path, reads the file and passes the contents to parse_raw. If content_type is omitted, it is inferred from the file's extension.

С помощью pydantic можно легко легко преобразовать dataclass в dataclass pydantic, который будет валидироваться

In [68]:
from dataclasses import dataclass

@dataclass
class User:
    name: Optional[str]
    email: str
    age: int

User(email='foo@bar.com', age='wrong data', name='foo')

User(name='foo', email='foo@bar.com', age='wrong data')

In [69]:
from pydantic.dataclasses import dataclass

@dataclass
class User:
    name: Optional[str]
    email: str
    age: int

User(email='foo@bar.com', age='wrong data', name='foo')

ValidationError: 1 validation error for User
age
  value is not a valid integer (type=type_error.integer)

Вложенные объекты данных намного проще описывать, чем в marshmallow

In [62]:
from pydantic import BaseModel
from typing import List

class Book(BaseModel):
    name: str
    isbn: int
        

class User(BaseModel):
    name: Optional[str]
    email: str
    books: List[Book]

        
User(**{
    'name': 'foo',
    'email': 'foo@bar.com',
    'books': [
        {
            'name': 'Foundation',
            'isbn': 1,
        },
        {
            'name': 'Foundation 2',
            'isbn': 2,
        },
    ],
})

User(name='foo', email='foo@bar.com', books=[Book(name='Foundation', isbn=1), Book(name='Foundation 2', isbn=2)])

#### Типы данных для валидации (Field types)

Pydantic поддерживает стандартные типы данных языка Python, типы и конструкции из модуля typing

In [7]:
from typing import Optional
from pydantic import BaseModel, EmailStr


class UserSchema(BaseModel):
    name: Optional[str]
    email: EmailStr

        
UserSchema(**{
    'name': 'foo',
    'email': 'foo@bar.com',
})

UserSchema(name='foo', email='foo@bar.com')

In [8]:
UserSchema(**{
    'name': None,
    'email': 'foo@bar.com',
})

UserSchema(name=None, email='foo@bar.com')

Union

In [65]:
from typing import Union
from pydantic import BaseModel


class FilmSchema(BaseModel):
    title: Union[int, str]

        
FilmSchema(**{
    'title': 'Star Wars',
})

FilmSchema(title='Star Wars')

In [66]:
FilmSchema(**{
    'title': 21,
})

FilmSchema(title=21)

Если написать:

In [70]:
from typing import Union
from pydantic import BaseModel


class FilmSchema(BaseModel):
    title: Union[str, int]

То для второго примера pydantic приведет 21 к строке, потому-что Pydantic  идет последовательно по типам из Union и пытается привести данные к этому типу. 
(tries to match types in the order of the union.) 
https://github.com/samuelcolvin/pydantic/issues/2513
https://github.com/PrettyWood/pydantic/pull/66/files#diff-bd467a363c9f155a2e6a7716aeca0c351089135a53d09a66138cea4f16506ebdR2728-R2738

In [71]:
FilmSchema(**{
    'title': 21,
})

FilmSchema(title='21')

Подробнее о field types можно прочитать в документации https://pydantic-docs.helpmanual.io/usage/types/

#### Custom types

Класс положительных чисел:

In [93]:
class PositiveNumber(int):
    @classmethod
    def validate(cls, v):
        if not isinstance(v, int):
            raise ValueError(f'int type is expected instead of {type(v)} type')
        if v < 0:
            raise ValueError(f'The number {v} is not positive')
        return v
            
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

In [96]:
from pydantic import BaseModel


class Square(BaseModel):
    side_length: PositiveNumber
        
Square(side_length=2)

Square(side_length=2)

In [97]:
Square(side_length=-2)

ValidationError: 1 validation error for Square
side_length
  The number -2 is not positive (type=value_error)

#### constraining  types

Допустим нам не нужно создавать свой тип данных, нас устроит обычный тип данных, но в них должно быть кастомное ограничение

Значения различных стандартных типов может быть ограничены используя функции con* типа. 
Например:
 * constr для str
 * conint для int
 * conlist для list
 * conset для set
 * etc


Попробуем ограничить значение стороны квадрата. Строна квадрата не может быть отрицательной. Так же в данном случае сделаем ее меньше 100

In [99]:
from pydantic import BaseModel, conint


class Square(BaseModel):
    side_length: conint(ge=0, lt=100)
        
Square(side_length=2)

Square(side_length=2)

In [100]:
Square(side_length=0)

Square(side_length=0)

In [101]:
Square(side_length=100)

ValidationError: 1 validation error for Square
side_length
  ensure this value is less than 100 (type=value_error.number.not_lt; limit_value=100)

In [102]:
Square(side_length=-2)

ValidationError: 1 validation error for Square
side_length
  ensure this value is greater than or equal to 0 (type=value_error.number.not_ge; limit_value=0)

#### Валидация данных с помощью pydantic

In [9]:
!pip install email-validator

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [79]:
from pydantic import BaseModel, constr, EmailStr
from typing import List

class BookSchema(BaseModel):
    name: constr(min_length=2, max_length=120)
    isbn: int
        

class UserSchema(BaseModel):
    name: Optional[str]
    email: EmailStr
    books: List[BookSchema]

        
UserSchema(**{
    'name': 'foo',
    'email': 'foo@bar.com',
    'books': [
        {
            'name': 'Foundation',
            'isbn': 1,
        },
        {
            'name': 'Foundation 2',
            'isbn': 2,
        },
    ],
})

UserSchema(name='foo', email='foo@bar.com', books=[BookSchema(name='Foundation', isbn=1), BookSchema(name='Foundation 2', isbn=2)])

#### Validator

In [104]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class GuestSchema(BaseModel):
    id: UUID
    name: str
    
    @validator('name')
    def no_dark_side_of_the_force(cls, value: str) -> str:
        if 'darth' in value.lower() or 'vaider' in value.lower():
            raise ValueError('Darth Vaider is not allowed!')
        return value

In [105]:
UserSchema(**{'id': uuid4(), 'name': 'Darth Vaider'})

ValidationError: 1 validation error for UserSchema
name
  Darth Vaider is not allowed! (type=value_error)

In [106]:
UserSchema(**{'id': uuid4(), 'name': 'Princess Leia'})

UserSchema(id=UUID('dc3424ef-fa6e-4ef5-a925-0c6ca4dd0492'), name='Princess Leia')

In [44]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class GuestSchema(BaseModel):
    id: UUID
    name: str
    password: str
    
    @validator('password')
    def validate_password(cls, value: str) -> str:
        if value != 'May the force be with you':
            raise ValueError('The password is incorrect')
        return value

In [45]:
GuestSchema(id=uuid4(), name='Princess Leia', password='May the force be with you')

GuestSchema(id=UUID('8dcc066f-aeb2-42a3-b8d3-fec3497c1349'), name='Princess Leia', password='May the force be with you')

In [46]:
GuestSchema(id=uuid4(), name='Darth Vaider', password='sniffing sounds')

ValidationError: 1 validation error for GuestSchema
password
  The password is incorrect (type=value_error)

### Validate 2 fields together

In [107]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class GuestSchema(BaseModel):
    id: UUID
    name: str
    password: str
    
    @validator('password', pre=True, always=True)
    def validate_password(cls, value, values) -> str:
        if value != 'May the force be with you':
            raise ValueError('The password is incorrect')
        if 'darth' in values.get('name').lower() or 'vaider' in values.get('name').lower():
            raise ValueError('The password is incorrect')
        return value

In [108]:
GuestSchema(id=uuid4(), name='Princess Leia', password='May the force be with you')

GuestSchema(id=UUID('8072ba0f-473a-447a-a780-e9276c65a9fa'), name='Princess Leia', password='May the force be with you')

In [109]:
GuestSchema(id=uuid4(), name='Darth Vaider', password='May the force be with you')

ValidationError: 1 validation error for GuestSchema
password
  The password is incorrect (type=value_error)

In [110]:
GuestSchema(id=uuid4(), name='Darth Vaider', password='May the force be with you!')

ValidationError: 1 validation error for GuestSchema
password
  The password is incorrect (type=value_error)

#### ValidationError

Pydantic поднимает ValidationError всякий раз, когда обнаруживает ошибку в проверяемых данных

In [82]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
        

User(name=[])

ValidationError: 1 validation error for User
name
  str type expected (type=type_error.str)

Валидаторы реализованные программистом, не должны поднимать ValidationError сами, вместо этого нужно использовать ValueError, TypeError или AssertionError (или подклассы ValueError или TypeError), так как эти исключения pydantic словит и будет использовать для ValidationError.

В конечном счете одно исключение будет вызвано и ValidationError будет содержать информацию обо всех ошибках и описание как они произошли.

Рассмотрим как можно получить доступ к ошибкам.

e.json() - возвращает ошибки в виде json (Удобно в web-приложениях, pydantic используется в fast-API)

In [88]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
        

try:
    User(name=[])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "name"
    ],
    "msg": "str type expected",
    "type": "type_error.str"
  }
]


e.errors() - возвращает список найденных ошибок во входных данных

In [89]:
try:
    User(name=[])
except ValidationError as e:
    print(e.errors())

[{'loc': ('name',), 'msg': 'str type expected', 'type': 'type_error.str'}]


str(e) - возвращает ошибки в читаемом формате

In [90]:
try:
    User(name=[])
except ValidationError as e:
    print(e)    # print(str(e))

1 validation error for User
name
  str type expected (type=type_error.str)


#### Export data

In [48]:
leia = GuestSchema(id=uuid4(), name='Princess Leia', password='May the force be with you')

In [49]:
leia.dict()

{'id': UUID('405f70db-591f-48c3-8461-e41cc8f3fc27'),
 'name': 'Princess Leia',
 'password': 'May the force be with you'}

In [50]:
leia.json()

'{"id": "405f70db-591f-48c3-8461-e41cc8f3fc27", "name": "Princess Leia", "password": "May the force be with you"}'

exclude

In [51]:
leia.json(exclude={'password'})

'{"id": "405f70db-591f-48c3-8461-e41cc8f3fc27", "name": "Princess Leia"}'

In [52]:
leia.dict(exclude={'password'})

{'id': UUID('405f70db-591f-48c3-8461-e41cc8f3fc27'), 'name': 'Princess Leia'}

#### Allow extra data

Иногда из сервисов можно получать избыточные данные. В таком случае можно простo использовать параметр extra='allow'


In [73]:
from pydantic import BaseModel
from uuid import UUID, uuid4


class User(BaseModel):
    id: UUID
    username: str
        
    class Config:
        extra='allow'

In [74]:
User(**{'username': 'admin', 'id': uuid4(), 'age': 30, 'location': 'Moscow'})

User(id=UUID('aa8c98ed-f479-4a36-9afa-0319bba06b46'), username='admin', age=30, location='Moscow')

In [77]:
from pydantic import BaseModel
from uuid import UUID, uuid4


class User(BaseModel):
    id: UUID
    username: str
        
    class Config:
        extra='ignore'

In [78]:
User(**{'username': 'admin', 'id': uuid4(), 'age': 30, 'location': 'Moscow'})

User(id=UUID('c0d189a1-1815-445c-98ce-84f711a34fb1'), username='admin')

#### Aliases
alias - указывает pydantic по какому ключу искать данные

In [19]:
import datetime

from typing import Union
from pydantic import BaseModel, Field



class PostSchema(BaseModel):
    user: str
    text: str
    last_modified_data: datetime.datetime = Field(alias='LastModifiedData')

In [21]:
post = {
    'user': 'admin',
    'text': 'Hello word!',
    'LastModifiedData': datetime.datetime(year=2021, month=5, day=4)
}

PostSchema(**post)

PostSchema(user='admin', text='Hello word!', last_modified_data=datetime.datetime(2021, 5, 4, 0, 0))

### Inheritance


Наследование работает так же как в python

In [19]:
from typing import Optional
from pydantic import BaseModel, EmailStr


class Person(BaseModel):
    email: EmailStr
        

class Customer(Person):
    total_spending: Optional[int] = 0

In [15]:
Customer(email='foo@bar.com', total_spending=200)

Customer(email='foo@bar.com', total_spending=200)

In [16]:
Customer('foo@bar.com', 200)

TypeError: __init__() takes exactly 1 positional argument (3 given)

In [20]:
Customer.__fields__

{'email': ModelField(name='email', type=EmailStr, required=True),
 'total_spending': ModelField(name='total_spending', type=Optional[int], required=False, default=0)}

In [21]:
from typing import Optional
from pydantic import BaseModel, EmailStr


class Person(BaseModel):
    email: EmailStr
    name: Optional[str]
        

class Customer(Person):
    total_spending: int

In [18]:
Customer(email='foo@bar.com', total_spending=200)

Customer(email='foo@bar.com', name=None, total_spending=200)

In [22]:
Customer('foo@bar.com', 200)

TypeError: __init__() takes exactly 1 positional argument (3 given)

Pydantic & Settings 

In [2]:
%set_env APP_HOST=127.0.0.1

env: APP_HOST=127.0.0.1


In [3]:
%set_env APP_PORT=8000

env: APP_PORT=8000


In [ ]:
!set

In [5]:
from pydantic import BaseSettings


class AppSettings(BaseSettings):
    host: str
    port: int
        
    class Config:
        env_prefix = 'APP_'

In [7]:
app_settings = AppSettings()
app_settings

AppSettings(host='127.0.0.1', port=8000)

In [8]:
app_settings.host

'127.0.0.1'

In [58]:
!unset APP_HOST

In [59]:
!unset APP_PORT

#### Inside Pydantic
Pydantic основан на:
 * \_\_annotations\_\_
 * Metaclasses
 * setattr

## Плюсы pydantic

## фичи

Optional: